In [ ]:
# # Place the triples folder under the directory!

# !git clone https://github.com/ForeverHaibara/Triple-SOS.git
# !cp Triple-SOS/triples triples

!pip install sympy>=1.10 --user
!pip install scipy>=1.6 --user
!pip install numpy --user
!pip install clarabel --user  # if not successful, install cvxopt instead
# please RESTART the kernel after installation

# Triples

**GitHub** https://github.com/ForeverHaibara/Triple-SOS

<!-- Symbolic computation is based on SymPy. -->

The package helps prove algebraic inequalities via exact sum-of-squares.

In [1]:
import sys
sys.path.append('../') # for triples

from triples.utils import *
# from triples.core import sum_of_squares
from triples.core import *
from IPython.display import display
import sympy as sp
from sympy import sqrt, Rational
import numpy as np
a, b, c, d, e, f, g, h, i, j, k, l, m, n, o, p, q, r, s, t, u, v, w, x, y, z = sp.symbols('a:z') # define SymPy symbols

## Basic Usage

Use `sum_of_squares(expr, ineq_constraints=[], eq_constraints=[])` to prove `expr >= 0`.

`Ineq_constraints` and `eq_constraints` are lists of symbolic expressions that are nonnegative and zero, respectively.

If it fails, it returns None. If it succeeds, it returns a Solution object.

The solution expression can be accessed by `.solution` property.

<!-- `sum_of_squares(expr: Expr, ineq_constraints: Union[List[Expr], Dict[Expr, Expr]], eq_constraints: Union[List[Expr], Dict[Expr, Expr]])` -->

#### Example: Nesbitt's Inequality

Goal: Prove that $\frac{a}{b+c}+\frac{b}{c+a}+\frac{c}{a+b} - \frac{3}{2}\geq 0$ given $a\geq 0,b\geq 0,c\geq 0$.

In [2]:
# Proving a/(b+c) + b/(c+a) + c/(a+b) >= 3/2 given a>=0, b>=0, c>=0
sol = sum_of_squares(CyclicSum(a/(b+c), (a,b,c)) - Rational(3,2), [a,b,c])
print(sol)
sol

Solution(problem = <InequalityProblem of 3 variables, with 3 inequality and 0 equality constraints>, solution = ((Σ(a*(b - c)**2))/2 + (Σ(a*(-2*a + b + c)**2))/2)/(2*(a + b)*(a + c)*(b + c)))


Solution(problem = <InequalityProblem of 3 variables, with 3 inequality and 0 equality constraints>, solution = ((Σ(a*(b - c)**2))/2 + (Σ(a*(-2*a + b + c)**2))/2)/(2*(a + b)*(a + c)*(b + c)))

In [3]:
sol = sum_of_squares(CyclicSum(a/(b+c), (a,b,c)) - Rational(3,2), [a,b,c])
print('Type of sol.solution =', type(sol.solution))

display(sol.solution) # the original solution is equal to this
display(sol.solution.doit()) # expand the cyclic sums

# Verify the result is correct (of course it is unless there is a bug)
diff = sol.solution - (a/(b+c)+b/(c+a)+c/(a+b) - Rational(3,2))
print('Is equal =', diff.simplify() == 0)

Type of sol.solution = <class 'sympy.core.mul.Mul'>


((Σ(a*(b - c)**2))/2 + (Σ(a*(-2*a + b + c)**2))/2)/(2*(a + b)*(a + c)*(b + c))

(a*(b - c)**2/2 + a*(-2*a + b + c)**2/2 + b*(-a + c)**2/2 + b*(a - 2*b + c)**2/2 + c*(a - b)**2/2 + c*(a + b - 2*c)**2/2)/(2*(a + b)*(a + c)*(b + c))

Is equal = True


In [ ]:
# Another display format: cancel the denominator:
sol = sum_of_squares(CyclicSum(a/(b+c), (a,b,c)) - Rational(3,2), [a,b,c])
sol.as_eq(cancel=True) # this is a sympy Equality object

Eq((a + b)*(a + c)*(b + c)*(Σ(a/(b + c)) - 3/2), (Σ(a*(b - c)**2))/4 + (Σ(a*(-2*a + b + c)**2))/4)


#### Example: Schur's inequality

Goal: Prove that $x^4(x-y)(x-z)+y^4(y-z)(y-x)+z^4(z-x)(z-y)\geq 0$ for all real numbers $x,y,z$.

In [ ]:
# Proving Schur's inequality of degree 6 for real numbers x,y,z. There are no inequality or equality constraints.

sum_of_squares(x**4*(x-y)*(x-z) + y**4*(y-z)*(y-x) + z**4*(z-x)*(z-y)).as_eq(cancel=True)

# sum_of_squares(x**4*(x-y)*(x-z) + y**4*(y-z)*(y-x) + z**4*(z-x)*(z-y)).doit() # Try using .doit() to expand cyclic sums

Solution(problem = <InequalityProblem of 3 variables, with 0 inequality and 0 equality constraints>, solution = ((4*(Σ(x))**2/3 + Σ((x - y)**2))*(∏((x - y)**2)) + (Σ((x - y)**2*(-3*x**3 + x**2*y + 2*x**2*z + x*y**2 - 4*x*y*z + 3*x*z**2 - 3*y**3 + 2*y**2*z + 3*y*z**2 - 2*z**3)**2))/9)/(Σ((x - y)**2)))

In [6]:
# get the tex code of the solution using `.to_string()`
sol = sum_of_squares(x**4*(x-y)*(x-z) + y**4*(y-z)*(y-x) + z**4*(z-x)*(z-y))
string = sol.to_string()
print(string)

print('='*80+'\n'+'='*80)
string2 = sol.doit().to_string() # the string after expanding cyclic sums
print(string2)

x^{4} \left(x - y\right) \left(x - z\right) + y^{4} \left(- x + y\right) \left(y - z\right) + z^{4} \left(- x + z\right) \left(- y + z\right) = \frac{\left(\frac{4 \left(\sum_{\mathrm{cyc}} x\right)^{2}}{3} + \sum_{\mathrm{cyc}} \left(x - y\right)^{2}\right) \prod_{\mathrm{cyc}} \left(x - y\right)^{2} + \frac{\sum_{\mathrm{cyc}} \left(x - y\right)^{2} \left(- 3 x^{3} + x^{2} y + 2 x^{2} z + x y^{2} - 4 x y z + 3 x z^{2} - 3 y^{3} + 2 y^{2} z + 3 y z^{2} - 2 z^{3}\right)^{2}}{9}}{\sum_{\mathrm{cyc}} \left(x - y\right)^{2}}
x^{4} \left(x - y\right) \left(x - z\right) + y^{4} \left(- x + y\right) \left(y - z\right) + z^{4} \left(- x + z\right) \left(- y + z\right) = \frac{\left(- x + z\right)^{2} \left(x - y\right)^{2} \left(y - z\right)^{2} \left(\left(- x + z\right)^{2} + \left(x - y\right)^{2} + \left(y - z\right)^{2} + \frac{4 \left(x + y + z\right)^{2}}{3}\right) + \frac{\left(- x + z\right)^{2} \left(- 3 x^{3} + 2 x^{2} y + x^{2} z + 3 x y^{2} - 4 x y z + x z^{2} - 2 y^{3} + 3 y^{2}

#### Example: 2000 IMO P2

Goal1: (2000 IMO P2) Given $a,b,c\geq 0$, $abc=1$, prove that $(a-1+\frac 1b)(b-1+\frac 1c)(c-1+\frac 1a)\leq 1$.

Goal2: (1983 IMO P6) Given $a,b,c$ be the sides of a triangle, prove that $a^2b(a-b)+b^2c(b-c)+c^2a(c-a)\geq 0$.

In [ ]:
# use ".problem" to access the whole problem, the problem class has a pleasant
# display format in Jupyter notebooks
sol = sum_of_squares(a**2*b*(a-b)+b**2*c*(b-c)+c**2*a*(c-a), [b+c-a, c+a-b, a+b-c])
problem = sol.problem
print(f"Problem Expression: {problem.expr}")
print(f"Inequality Constraints: {problem.ineq_constraints}")
print(f"Equality Constraints: {problem.eq_constraints}")
problem

Problem: a**2*b*(a - b) + a*c**2*(-a + c) + b**2*c*(b - c)
Inequality Constraints: {-a + b + c: -a + b + c, a - b + c: a - b + c, a + b - c: a + b - c}
Equality Constraints: {}


<InequalityProblem of 3 variables, with 3 inequality and 0 equality constraints>

### Properties and Methods of `Solution` class objects:

* `.solution` returns the SymPy expression, which is equal to the input expression
* `.problem` accesses the input inequality problem
* `.time` returns the solving time
* `.as_eq()` returns a SymPy Equality object, which has `.lhs` and `.rhs` properties.
* `.to_string()` returns the string
* `.doit()` returns a new Solution object with cyclic expressions expanded
* `.xreplace(...)` returns a new Solution by applying `xrepalce` on the solution
* (More to be added)

## Using Alias

Use a python dictionary rather a list to provide the "alias" of a constraint. This helps track the constraints.
It works for both inequality and equality constraints.

In [8]:
# Proving 5 - 3x - 4y >= 0 given no inequality constraint and one equality constraint x^2+y^2=1:
display(sum_of_squares(5 - 3*x - 4*y, [], [x**2+y**2-1]))

# We can also let x^2+y^2-1 = a = 0 by using a dictionary:
display(sum_of_squares(5 - 3*x - 4*y, [], {x**2+y**2-1: a}))

# Let it be a symbol called "(x^{2}+y^{2}-1)", this avoids expanding the terms:
display(sum_of_squares(5 - 3*x - 4*y, [], {x**2+y**2-1: sp.Symbol('(x^{2}+y^{2}-1)')}))

Solution(problem = <InequalityProblem of 2 variables, with 0 inequality and 1 equality constraints>, solution = -5*x**2/2 - 5*y**2/2 + (5*x - 3)**2/10 + (5*y - 4)**2/10 + 5/2)

Solution(problem = <InequalityProblem of 2 variables, with 0 inequality and 1 equality constraints>, solution = -5*a/2 + (5*x - 3)**2/10 + (5*y - 4)**2/10)

Solution(problem = <InequalityProblem of 2 variables, with 0 inequality and 1 equality constraints>, solution = -5*(x^{2}+y^{2}-1)/2 + (5*x - 3)**2/10 + (5*y - 4)**2/10)

In [9]:
# Proving CyclicSum(a**2*(a+b)*(b-c), (a,b,c)) >= 0 given b+c-a>=0, c+a-b>=0, a+b-c >= 0
# We also let F(a), F(b), F(c) = b+c-a, c+a-b, a+b-c by using a dictionary:
F = sp.Function('F')
sum_of_squares(CyclicSum(a**2*(a+b)*(b-c), (a,b,c)), {b+c-a:F(a), c+a-b:F(b), a+b-c:F(c)})

Solution(problem = <InequalityProblem of 3 variables, with 3 inequality and 0 equality constraints>, solution = (Σ(((-a + b)**2*F(a)*F(c) + 2*(a**2 - 2*a*c + b*c)**2)*F(b)))/(2*(Σ(F(a)))))

## More Advanced Features

More advanced features:

* Squareroots and other algebraic operators (sympy.Abs/Max/Min) are supported.

* Sines, cosines and other trignometric functions are supported if it is algebraic after a change of the variables.

These features are still under active development and are very likely to fail for difficult problems. It is recommended to simplify the problem manually if you can.

In [10]:
# Proving x^3+y^3+z^3-(x^2y+y^2z+z^2x) + (sqrt(13+16sqrt(2))-1)/2*(x-y)(y-z)(z-x) >= 0 for x,y,z >= 0
problem = CyclicSum(x**3,(x,y,z))-CyclicSum(x**2*y,(x,y,z)) + (sqrt(13+16*sqrt(2))-1)/2*(x-y)*(y-z)*(z-x)
sum_of_squares(problem, [x,y,z]) # hint: use .doit() to expand cyclic sums

Solution(problem = <InequalityProblem of 3 variables, with 3 inequality and 0 equality constraints>, solution = (2*(∏(x))*(Σ((x - y)**2)) + (Σ(x*(14*y**2 + y*(-x + z)*(-3*sqrt(13 + 16*sqrt(2)) + 7 + sqrt(2)*sqrt(13 + 16*sqrt(2)) + 7*sqrt(2)) - 14*z**2 + z*(x - y)*(-sqrt(2)*sqrt(13 + 16*sqrt(2)) + 7 + 7*sqrt(2) + 3*sqrt(13 + 16*sqrt(2))))**2))/98)/(2*(Σ(x*y))))

In [11]:
# Proving a*sqrt(4*a**2+5*b*c) + b*sqrt(4*b**2+5*c*a) + c*sqrt(4*c**2+5*a*b) >= (a+b+c)^2 for a,b,c >= 0
problem = a*sqrt(4*a**2+5*b*c) + b*sqrt(4*b**2+5*c*a) + c*sqrt(4*c**2+5*a*b) - (a+b+c)**2
sol = sum_of_squares(problem, [a,b,c])
print('Success =', sol is not None)
display(sol)

# Verify sol.solution == problem when (a,b,c) = (1,4,7) numerically (20 digits):
# (expanding might take too much time)
print('Problem.subs({a:1,b:4,c:7}) =', problem.subs({a:1,b:4,c:7}).simplify().n(20))
print('Solution.subs({a:1,b:4,c:7}) =', sol.solution.subs({a:1,b:4,c:7}).simplify().n(20))

Success = True


Solution(problem = <InequalityProblem of 3 variables, with 3 inequality and 0 equality constraints>, solution = (12478632*(Σ((a - b)**2*(4*a**2*b**2 + a*b*(c - sqrt(5*a*c + 4*b**2))**2))) + (Σ((sqrt(4*a**2 + 5*b*c) - sqrt(5*a*c + 4*b**2))**2*(4291307*a*b**2*sqrt(4*a**2 + 5*b*c) + 9265396*a*b*sqrt(4*a**2 + 5*b*c)*sqrt(5*a*c + 4*b**2) + 1903200*a*b*sqrt(5*a*b + 4*c**2)*sqrt(5*a*c + 4*b**2) + 3007056*b*sqrt(4*a**2 + 5*b*c)*(-b + c)**2 + 133224*b*sqrt(5*a*c + 4*b**2)*(sqrt(5*a*b + 4*c**2) - sqrt(5*a*c + 4*b**2))**2)))/2 + (Σ(a*(-sqrt(4*a**2 + 5*b*c) + sqrt(5*a*c + 4*b**2))**2*(16687876*b**2*c + 6983095*b*c*sqrt(4*a**2 + 5*b*c) + 15320728*b*(-a + b)**2)))/2 + (Σ(a*b*(-3*c + sqrt(4*a**2 + 5*b*c))**2*(16144582*c**2 + 19213793*c*sqrt(4*a**2 + 5*b*c))))/2 + 3013400*(Σ(b*c**2*sqrt(4*a**2 + 5*b*c)*(b - c)**2)) + 3800874*(Σ(b*c**2*(b - c)**2*sqrt(5*a*c + 4*b**2))))/(Σ(1421056*a**4 + 24360896*a**3*b + 355264*a**3*sqrt(4*a**2 + 5*b*c) + 57425824*a**2*b**2 + 78723325*a**2*b*c + 35310640*a**2*b*sqrt(4

Problem.subs({a:1,b:4,c:7}) = 10.678066681158278314
Solution.subs({a:1,b:4,c:7}) = 10.678066681158278314


In [12]:
# Proving sin(x)+sin(2x)+sin(3x) <= 5/2 given a real number x:
problem = Rational(5,2) - sp.sin(x) - sp.sin(2*x) - sp.sin(3*x)
sol = sum_of_squares(problem)
print('Success =', sol is not None)
display(sol.as_eq())

# Verify sol.solution == problem when x = 2 numerically (20 digits):
# (simplification might take too much time)
print('Problem.subs({x:2}) =', problem.subs({x:2}).simplify().n(20))
print('Solution.subs({x:2}) =', sol.solution.subs({x:2}).simplify().n(20))

Success = True


Eq(-sin(x) - sin(2*x) - sin(3*x) + 5/2, (2*(-sin(x) + cos(x))**4 + 8*(-sin(x) + cos(x)**2)**2)/(2*((cos(x) - 1)**2/2 + (cos(x) + 1)**2/2 + sin(x)**2)))

Problem.subs({x:2}) = 2.6269205666811724288
Solution.subs({x:2}) = 2.6269205666811724288
